In [1]:
import mysql.connector as db
import pandas as pd

In [2]:
user = "ShopEasy"
passcode="Visva1610@"
host="localhost"
db_name = "shopeasy"

In [3]:
# Connection
db_connection = db.connect(
    host=host,
    user=user,
    password=passcode,
    database=db_name
)

In [4]:
#defining cursor

curr = db_connection.cursor()

In [18]:
customer_journey = pd.read_csv("customer_journey.csv")

In [20]:
customer_reviews = pd.read_csv("customer_reviews.csv")

In [22]:
customers = pd.read_csv("customers.csv")

In [24]:
engagement_data = pd.read_csv("engagement_data.csv")

In [26]:
geography = pd.read_csv("geography.csv")

In [28]:
products = pd.read_csv("products.csv")

In [30]:
customer_journey["Duration"] = customer_journey["Duration"].fillna(0)

In [34]:
# create table - geography
sql = """
    create table geography(
    GeographyID int primary key,
    Country varchar(60),
    City varchar(60)
    )

"""
curr.execute(sql)

In [48]:
for index,row in geography.iterrows():
    sql="""
        insert into geography(GeographyID,Country,City) values (%s,%s,%s)
    
    """

    values = (row["GeographyID"],row["Country"],row["City"])

    curr.execute(sql,values)
    db_connection.commit()


In [36]:
# Create Table - customers

sql = """
    create table customers(
    CustomerID int PRIMARY KEY,
    CustomerName varchar(70),
    Email varchar(150),
    Gender varchar(10),
    Age int,
    GeographyID int,
    FOREIGN KEY (GeographyID) REFERENCES geography(GeographyID)
    );

"""
curr.execute(sql)

In [50]:
for index,row in customers.iterrows():
    sql="""
        insert into customers(CustomerID,CustomerName,Email,Gender,Age,GeographyID) values (%s,%s,%s,%s,%s,%s)
    
    """

    values = (row["CustomerID"],row["CustomerName"],row["Email"],row["Gender"],row["Age"],row["GeographyID"])

    curr.execute(sql,values)
    db_connection.commit()

In [38]:
# create table - products
sql = """
    create table products(
    ProductID int PRIMARY KEY,
    ProductName varchar(100),
    Category varchar(60),
    Price float
    )

"""
curr.execute(sql)

In [52]:
for index,row in products.iterrows():
    sql="""
        insert into products(ProductID,ProductName,Category,Price) values (%s,%s,%s,%s)
    
    """

    values = (row["ProductID"],row["ProductName"],row["Category"],row["Price"])

    curr.execute(sql,values)
    db_connection.commit()

In [62]:
## Create Table - Customer_Journey
sql = """

create table customer_journey(
JourneyID int primary key,
CustomerID int ,
ProductID int,
VisitDate date,
Stage varchar(30),
Action varchar(30),
Duration float,
foreign key (CustomerID) references customers(CustomerID),
FOREIGN KEY (ProductID) REFERENCES products(ProductID)
);

"""
curr.execute(sql)

In [64]:
# inserting values in customer_journey Table

for index,row in customer_journey.iterrows():
    sql="""
        insert into customer_journey(JourneyID, CustomerID, ProductID, VisitDate, Stage, Action, Duration) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """

    values=(row["JourneyID"],row["CustomerID"],row["ProductID"],row["VisitDate"],row["Stage"],row["Action"],row["Duration"])
    curr.execute(sql,values)
    db_connection.commit()

In [42]:
# create table - customer_reviews
sql = """
    create table customer_reviews(
    ReviewID int PRIMARY KEY,
    CustomerID int,
    ProductID int,
    ReviewDate date,
    Rating int,
    ReviewText varchar(500),
    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
    FOREIGN KEY (ProductID) REFERENCES products(ProductID)
    )

"""
curr.execute(sql)

In [66]:
for index,row in customer_reviews.iterrows():
    sql="""
        insert into customer_reviews(ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText) values (%s,%s,%s,%s,%s,%s)
    
    """

    values = (row["ReviewID"],row["CustomerID"],row["ProductID"],row["ReviewDate"],row["Rating"],row["ReviewText"])

    curr.execute(sql,values)
db_connection.commit()

In [44]:
# create table - engagement_data
sql = """
    create table engagement_data(
    EngagementID int PRIMARY KEY,
    ContentID int,
    ContentType varchar(60),
    Likes int,
    EngagementDate date,
    CampaignID int,
    ProductID int,
    ViewsClicksCombined varchar(70),
    FOREIGN KEY (ProductID) REFERENCES products(ProductID)
    )

"""
curr.execute(sql)

In [68]:
for index,row in engagement_data.iterrows():
    sql="""
        insert into engagement_data(EngagementID,ContentID,ContentType,Likes,EngagementDate,CampaignID,ProductID,ViewsClicksCombined) values (%s,%s,%s,%s,%s,%s,%s,%s)
    
    """

    values = (row["EngagementID"],row["ContentID"],row["ContentType"],row["Likes"],row["EngagementDate"],row["CampaignID"],row["ProductID"],row["ViewsClicksCombined"])

    curr.execute(sql,values)
    db_connection.commit()


 ## Finding total customers at each stage

In [115]:
sql = """ select Stage,count( distinct CustomerID) as Customer from customer_journey group by Stage order by Customer DESC; """

curr.execute(sql)

In [117]:
TotalCustomerAtEachStage =pd.DataFrame(curr.fetchall(), columns=["Stage", "Customer"])

In [119]:
TotalCustomerAtEachStage

,Stage,Customer
0,Homepage,40
1,ProductPage,24
2,Checkout,19


In [121]:
# From the above results we can see there's a decline in Customer's purchase perspecrtive as he/she tried to move from Homepage to Checkout Page

## Average Duration at each Stage

In [126]:
sql = """select Stage,Round(AVG(Duration),2) as AVG_Duration from customer_journey group by Stage order by AVG_Duration DESC;"""

curr.execute(sql)
AvgDuration = pd.DataFrame(curr.fetchall(),columns=["Stage","AVG_Duration"])
AvgDuration

,Stage,AVG_Duration
0,ProductPage,182.77
1,Homepage,160.54
2,Checkout,45.05


In [128]:
# From the above results we can see customers are moving from HomePage to ProductPage 
# and people are viewing the product but due to high price or product reviews the duration spent in checkout page is less 
# because number of customers moving to checkout is less and people are dropping off at checkout page only, this may be due to 
# page interaction or additional charges or misleading info displayed in Checkout page etc...

## Finding Purchases and Dropoff Count

In [15]:
sql = """SELECT Stage,Action,COUNT(*) AS Action_Count
FROM customer_journey WHERE Stage = 'Checkout'
GROUP BY Stage, Action;"""
curr.execute(sql)
PurchaseandDropOffCount = pd.DataFrame(curr.fetchall(), columns=["Stage","Action","Action_Count"])
PurchaseandDropOffCount


,Stage,Action,Action_Count
0,Checkout,Drop-off,14
1,Checkout,Purchase,6


In [20]:
# From the above results out of 20 people 14 droped off at checkout and only 6 made a purchase that's 70% dropoff at the checkout stage alone.
# People are intrested to buy the products but due to additional charges or misleading informations 
# or delayed delivery dates or delivery not availble at their home location 70% of the people droped off their purchases.

# Solution:
# Provide some discounts for regular customer, display messages like "Get 10% of on your 3rd purchase"
# Increase Delivery Speed, try to provide free delivery for near by Loactions.
# Cancel additional fees like delivery fee if they have purchased more than a certain amount.
# Increase the range of delivery to all locations
# Make the checkout page more attractive, simple and easily understandable, avoid any misleading statements.

## Drop-offs by Products

In [18]:
sql = """SELECT p.ProductName,p.Price, COUNT(*) AS Drop_offs
FROM customer_journey j
JOIN products p ON j.ProductID = p.ProductID
WHERE j.Stage = 'Checkout' AND j.Action = 'Drop-off'
GROUP BY p.ProductName,p.Price
ORDER BY Drop_offs DESC;"""
curr.execute(sql)
DropOffCountbyProducts = pd.DataFrame(curr.fetchall(), columns=["ProductName","Price","Drop_offs"])
DropOffCountbyProducts


,ProductName,Price,Drop_offs
0,Ski Boots,340.20,3
1,Boxing Gloves,262.32,3
2,Volleyball,42.80,2
3,Ice Skates,37.56,1
4,Tennis Racket,36.07,1
5,Surfboard,275.43,1
6,Baseball Glove,327.36,1
7,Basketball,225.12,1
8,Football Helmet,44.75,1


## Purchase by Products

In [25]:
sql = """SELECT p.ProductName,p.Price, COUNT(*) AS PurchaseCount
FROM customer_journey j
JOIN products p ON j.ProductID = p.ProductID
WHERE j.Stage = 'Checkout' AND j.Action = 'Purchase'
GROUP BY p.ProductName,p.Price
ORDER BY PurchaseCount DESC;"""
curr.execute(sql)
PurchasebyProducts = pd.DataFrame(curr.fetchall(), columns=["ProductName","Price","PurchaseCount"])
PurchasebyProducts


,ProductName,Price,PurchaseCount
0,Surfboard,275.43,2
1,Climbing Rope,410.17,2
2,Fitness Tracker,196.68,1
3,Kayak,259.40,1


In [ ]:
# Even high priced products are bought so price alone is not the conversion factor
# Low Priced Products are also Droped off, We need to investigate Product Specifications, rating, additional Charges, Webpage UI/UX
# Surfboard and Climbing Rope have better conversion despite high prices.Focus on replicating their success factors(e.g.,marketing, product descriptions).


## Products analysis based on each location

In [30]:
sql = """SELECT
    g.Country,
    g.City,
    p.ProductName,
    p.Price,
    COUNT(DISTINCT CASE WHEN j.Action = 'Purchase' THEN j.JourneyID END) AS TotalPurchases,
    COUNT(DISTINCT CASE WHEN j.Action = 'Drop-off' THEN j.JourneyID END) AS TotalDropOffs,
    ROUND(SUM(CASE WHEN j.Action = 'Purchase' THEN p.Price ELSE 0 END), 2) AS TotalRevenue
FROM geography g
JOIN customers c ON g.GeographyID = c.GeographyID
JOIN customer_journey j ON c.CustomerID = j.CustomerID
JOIN products p ON j.ProductID = p.ProductID
WHERE j.Stage = 'Checkout'
GROUP BY g.Country, g.City, p.ProductName,p.Price
ORDER BY TotalPurchases DESC, TotalDropOffs DESC;"""
curr.execute(sql)
PurchasebyLocation = pd.DataFrame(curr.fetchall(), columns=["Country","City","ProductName","Price","TotalPurchases","TotalDropOffs","TotalRevenue"])
PurchasebyLocation


,Country,City,ProductName,Price,TotalPurchases,TotalDropOffs,TotalRevenue
0,Spain,Madrid,Surfboard,275.43,1,1,275.43
1,Austria,Vienna,Kayak,259.40,1,0,259.40
2,Germany,Berlin,Fitness Tracker,196.68,1,0,196.68
3,Netherlands,Amsterdam,Climbing Rope,410.17,1,0,410.17
4,Sweden,Stockholm,Climbing Rope,410.17,1,0,410.17
5,UK,London,Surfboard,275.43,1,0,275.43
6,Austria,Vienna,Boxing Gloves,262.32,0,1,0.00
7,Austria,Vienna,Ski Boots,340.20,0,1,0.00
8,Belgium,Brussels,Boxing Gloves,262.32,0,1,0.00
9,Belgium,Brussels,Ski Boots,340.20,0,1,0.00


In [ ]:
# Even high priced products like Surfboard and Climbing Rope are bought so use those campaigning technique in other locations too.
# Analyse the sports based on the locations geography and increase campaigns for those products, provide some deals/discounts too.

# Customer Review Analysis

### Overall Rating Distribution

In [15]:
sql = """SELECT Rating,COUNT(*) AS ReviewCount
FROM customer_reviews
GROUP BY Rating
ORDER BY Rating DESC;"""
curr.execute(sql)
OverallRating = pd.DataFrame(curr.fetchall(), columns=["Rating","ReviewCount"])
OverallRating


,Rating,ReviewCount
0,5,23
1,4,39
2,3,29
3,2,6
4,1,3


### Top 5 Best-Rated Products (with at least 3 reviews)

In [18]:
sql="""SELECT 
    p.ProductName,
    ROUND(AVG(r.Rating), 2) AS AvgRating,
    COUNT(r.ReviewID) AS ReviewCount
FROM 
    products p
JOIN 
    customer_reviews r ON p.ProductID = r.ProductID
GROUP BY 
    p.ProductName
HAVING 
    COUNT(r.ReviewID) >= 3
ORDER BY 
    AvgRating DESC
Limit 5;"""

curr.execute(sql)
BestRatedProducts = pd.DataFrame(curr.fetchall(), columns=["ProductName","AvgRating","ReviewCount"])
BestRatedProducts

,ProductName,AvgRating,ReviewCount
0,Football Helmet,5.00,3
1,Hockey Stick,4.40,5
2,Running Shoes,4.00,4
3,Soccer Ball,4.00,3
4,Ski Boots,4.00,6


### Top 5 Below Average-Rated Products

In [26]:
sql="""SELECT 
    p.ProductName,
    ROUND(AVG(r.Rating), 2) AS AvgRating,
    COUNT(r.ReviewID) AS ReviewCount
FROM 
    products p
JOIN 
    customer_reviews r ON p.ProductID = r.ProductID
GROUP BY 
    p.ProductName
HAVING 
    COUNT(r.ReviewID) >= 3
ORDER BY 
    AvgRating ASC
Limit 5;"""

curr.execute(sql)
BelowAverageRatedProducts = pd.DataFrame(curr.fetchall(), columns=["ProductName","AvgRating","ReviewCount"])
BelowAverageRatedProducts

,ProductName,AvgRating,ReviewCount
0,Basketball,2.67,3
1,Dumbbells,3.00,5
2,Baseball Glove,3.40,5
3,Kayak,3.40,10
4,Golf Clubs,3.50,4


### Top 5 Best-Rated Products with Customer Journey (with at least 3 reviews)

In [33]:
sql="""SELECT 
    p.ProductName,
    ROUND(AVG(r.Rating), 2) AS AvgRating,
    COUNT(r.ReviewID) AS ReviewCount,
    p.Price,
    IF(EXISTS (
        SELECT 1 
        FROM customer_journey j 
        WHERE j.ProductID = p.ProductID 
        AND j.Action = 'Purchase'
        LIMIT 1
    ), 'Yes', 'No') AS WasPurchased,
    (
        SELECT GROUP_CONCAT(DISTINCT CONCAT(j.Stage, ':', j.Action) SEPARATOR ', ')
        FROM customer_journey j 
        WHERE j.ProductID = p.ProductID
        LIMIT 100
    ) AS CustomerJourney
FROM 
    products p
JOIN 
    customer_reviews r ON p.ProductID = r.ProductID
GROUP BY 
    p.ProductID, p.ProductName, p.Price
HAVING 
    COUNT(r.ReviewID) >= 3
ORDER BY 
    AvgRating DESC
LIMIT 5;"""

curr.execute(sql)
BestRatedProducts = pd.DataFrame(curr.fetchall(), columns=["ProductName","AvgRating","ReviewCount","Price","WasPurchased","CustomerJourney"])
BestRatedProducts

,ProductName,AvgRating,ReviewCount,Price,WasPurchased,CustomerJourney
0,Football Helmet,5.00,3,44.75,No,"Checkout:Drop-off, Homepage:Click, Homepage:Vi..."
1,Hockey Stick,4.40,5,173.83,No,"homepage:Click, Homepage:View, ProductPage:Click"
2,Running Shoes,4.00,4,223.75,No,"Homepage:Click, Homepage:View, ProductPage:Cli..."
3,Soccer Ball,4.00,3,41.26,No,"Homepage:View, ProductPage:View"
4,Ski Boots,4.00,6,340.20,No,"Checkout:Drop-off, Homepage:Click, Homepage:View"


In [ ]:
# Customers are loving these products and they are wiling to buy it, but conversion didn't happen even though customers are intrested.
# They are giving more rating for the product quality, but according to my analysis demand for these products are more if we provide a discount
# the purchase rate for these products will increase.

### Top 5 Below Average-Rated Products with Customer Journey (with at least 3 reviews)

In [37]:
sql="""SELECT 
    p.ProductName,
    ROUND(AVG(r.Rating), 2) AS AvgRating,
    COUNT(r.ReviewID) AS ReviewCount,
    p.Price,
    IF(EXISTS (
        SELECT 1 
        FROM customer_journey j 
        WHERE j.ProductID = p.ProductID 
        AND j.Action = 'Purchase'
        LIMIT 1
    ), 'Yes', 'No') AS WasPurchased,
    (
        SELECT GROUP_CONCAT(DISTINCT CONCAT(j.Stage, ':', j.Action) SEPARATOR ', ')
        FROM customer_journey j 
        WHERE j.ProductID = p.ProductID
        LIMIT 100
    ) AS CustomerJourney
FROM 
    products p
JOIN 
    customer_reviews r ON p.ProductID = r.ProductID
GROUP BY 
    p.ProductID, p.ProductName, p.Price
HAVING 
    COUNT(r.ReviewID) >= 3
ORDER BY 
    AvgRating ASC
LIMIT 5;"""

curr.execute(sql)
BelowAverageRatedProducts = pd.DataFrame(curr.fetchall(), columns=["ProductName","AvgRating","ReviewCount","Price","WasPurchased","CustomerJourney"])
BelowAverageRatedProducts

,ProductName,AvgRating,ReviewCount,Price,WasPurchased,CustomerJourney
0,Basketball,2.67,3,225.12,No,Checkout:Drop-off
1,Dumbbells,3.00,5,26.21,No,"Homepage:Click, Homepage:View, ProductPage:View"
2,Baseball Glove,3.40,5,327.36,No,"Checkout:Drop-off, Homepage:Click, Homepage:Vi..."
3,Kayak,3.40,10,259.40,Yes,"Checkout:Purchase, Homepage:View, ProductPage:..."
4,Golf Clubs,3.50,4,81.59,No,"Homepage:Click, Homepage:View, ProductPage:View"


In [ ]:
# From the Top 5 Below Average rated products we can see customers are not satisfied with the product 
# because people stopped at product page only few went to checkout page.
# There maybe misleading informations about the product or product price maybe high or the demand for the product is less
# we can see there's one Purchase here, if we follow the marketing methods on other products we can get more conversion rate for other products too

# Marketing Effectiveness

###  Best-Performing Products by Region

In [40]:
sql="""   SELECT
    g.Country,
    g.City,
    p.ProductName,
    COUNT(DISTINCT CASE WHEN j.Action = 'Purchase' THEN j.JourneyID END) AS purchase_count,
    ROUND(SUM(CASE WHEN j.Action = 'Purchase' THEN p.Price ELSE 0 END), 2) AS total_revenue
FROM
    geography g
JOIN
    customers c ON g.GeographyID = c.GeographyID
JOIN
    customer_journey j ON c.CustomerID = j.CustomerID
JOIN
    products p ON j.ProductID = p.ProductID
WHERE
    j.Action = 'Purchase'
GROUP BY
    g.Country, g.City, p.ProductName
ORDER BY
    g.Country, g.City, purchase_count DESC;"""

curr.execute(sql)
BestProductsbyRegion = pd.DataFrame(curr.fetchall(), columns=["Country","City","ProductName","purchase_count","total_revenue"])
BestProductsbyRegion

,Country,City,ProductName,purchase_count,total_revenue
0,Austria,Vienna,Kayak,1,259.40
1,Germany,Berlin,Fitness Tracker,1,196.68
2,Netherlands,Amsterdam,Climbing Rope,1,410.17
3,Spain,Madrid,Surfboard,1,275.43
4,Sweden,Stockholm,Climbing Rope,1,410.17
5,UK,London,Surfboard,1,275.43


In [42]:
# The above results we can see the purchase count is only 1,we need to do valid campaigns and provide some discount
# on the products to increase the sales

### Calculate Customer Retention Rate 

In [49]:
sql= """WITH CustomerActivity AS (
    SELECT 
        CustomerID,
        COUNT(DISTINCT DATE(VisitDate)) AS active_months
    FROM 
        customer_journey
    WHERE 
        Action = 'Purchase'
    GROUP BY 
        CustomerID
    HAVING 
        COUNT(DISTINCT DATE(VisitDate)) >= 1
),
RetentionMetrics AS (
    SELECT
        COUNT(DISTINCT CASE WHEN active_months > 1 THEN CustomerID END) AS repeat_customers,
        COUNT(DISTINCT CustomerID) AS total_customers
    FROM 
        CustomerActivity
)
SELECT
    ROUND((repeat_customers * 100.0 / total_customers), 2) AS retention_rate_percentage
FROM
    RetentionMetrics;"""
curr.execute(sql)
RetentionRate = pd.DataFrame(curr.fetchall(),columns=["retention_rate_percentage"])
RetentionRate

,retention_rate_percentage
0,0.00


In [51]:
# There's no repeat Customers, we need to attract the customers who previously bought
# a product by providing an offer saying "Get some discount at your 2nd or 3rd purchase"
# We need to do more marketing and attract new buying customers

# Recommendations on improving customer experience

In [ ]:
# Simplify the Checkout process for High-Drop-Off Products
# Address Low-Rated Products, try to replace them with good quality and minimum priced products
# Personalize Recommendations by Region
# send out Discount Emails to the customers who dropped off at checkout stage

# Marketing Analysis

### Overall Performance by Content Type

In [60]:
sql =""" SELECT 
    LOWER(ContentType) AS ContentType,
    SUM(Likes) AS TotalLikes,
    SUM(SUBSTRING_INDEX(ViewsClicksCombined, '-', 1)) AS TotalViews,  
    SUM(SUBSTRING_INDEX(ViewsClicksCombined, '-', -1)) AS TotalClicks,
    ROUND(SUM(SUBSTRING_INDEX(ViewsClicksCombined, '-', -1)) * 100.0 / 
          SUM(SUBSTRING_INDEX(ViewsClicksCombined, '-', 1)), 2) AS ClickThroughRate
FROM 
    engagement_data
GROUP BY 
    LOWER(ContentType)
ORDER BY 
    TotalClicks DESC;"""
curr.execute(sql)
OverallPerformance = pd.DataFrame(curr.fetchall(),columns=["ContentType","TotalLikes","TotalViews","TotalClicks","ClickThroughRate"])
OverallPerformance

,ContentType,TotalLikes,TotalViews,TotalClicks,ClickThroughRate
0,blog,2999,65977.0,14300.0,21.67
1,video,3473,64073.0,13803.0,21.54
2,socialmedia,2594,59936.0,12498.0,20.85
3,newsletter,1380,37059.0,6350.0,17.13


In [62]:
## Blogs acts as a higher marketing platform, so we can post more product details and perform campaigns in blogs